Load one of each type of NN and RNN model for the graphs in our paper.

In [1]:
%run ../MLHCExperiment-Common-Val17Test18.ipynb

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
all_visits = read_data()
all_visits = all_visits.where(all_visits["AdjCount"] > 0).dropna()

In [3]:
def run_dynamic_rnn(filename, first_layer_size, second_layer_size, name, features, label):
    print(name)
    X = all_visits[features]
    y = all_visits[label]

    val_mses = list()
    mses = list()

    start = time.time()
    features_min_max = preprocessing.MinMaxScaler()
    X_train, y_train, X_val, y_val, X_test, y_test = split_for_baseline_and_nn(X, y)

    model = compile_rnn_weights_from_file(filename, X_train, first_layer_size, second_layer_size)

    X_train_t = reshape_for_rnn(features_min_max.fit_transform(X_train))
    X_val_t = reshape_for_rnn(features_min_max.transform(X_val))
    X_test_t = reshape_for_rnn(features_min_max.transform(X_test))

    y_val_pred = model.predict(X_val_t, batch_size=1)
    y_pred = model.predict(X_test_t, batch_size=1)
    val_mses.append(mean_squared_error(y_val_pred, y_val))
    mses.append(mean_squared_error(y_pred, y_test))
    end = time.time()
    print("Trial completed in %.2f s" % (end - start))
    
    return val_mses, mses, y_pred

In [4]:
for arch in possible_architectures_rnn:
    print(arch)
    filenames = prebuilt_rnn_models[arch]
    
    if arch == "32-32R":
        label = ICD8_LABEL
    elif arch == "32-32NR":
        label = ICD8OPP_LABEL
    else:
        label = ALL_VISITS_LABEL
    
    # run only the first file
    for filename in filenames:
        v_mses, mses, y_pred = run_dynamic_rnn("../" + filename, layer_sizes[arch][0], layer_sizes[arch][1],
                                       filename, BASE_FEATURES, label)
        
        print_mse_metrics(v_mses)
        print_mse_metrics(mses)
        print(list(y_pred.reshape(-1)))
        print("\n")
        break

32-32
RNNModels/32-32-0Lag-All/Dynamic-RNN-AllVisits-32-32-Trial0-1562316203.5722883.h5
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Trial completed in 3.53 s
MSE and RMSE over 10 trials with standard deviation in parentheses
Average MSE: 0.709 (0.000)
Average RMSE: 0.842 (0.000)
MSE and RMSE over 10 trials with standard deviation in parentheses
Average MSE: 1.094 (0.000)
Average RMSE: 1.046 (0.000)
[1.6810899, 7.7716665, 7.5157127, 1.0093269, 6.204715, 3.7312694, 1.860593, 10.086176, 8.403478, 8.011694, 7.863287, 7.818218, 3.5843158, 1.6959761, 10.528662, 8.65031, 8.044501, 7.5198736, 8.599273, 4.1477685, 2.0510144, 10.751688, 8.405675, 7.95098, 7.8774962, 8.24617, 4.018567, 1.8771431, 10.663866, 8.104735, 7.8746185, 8.13449, 8.301005, 4.0838337, 2.054473, 10.165032, 9.1218605, 8.139985, 7.624545, 8.036632, 4.315897, 2.0266883, 10.408492, 8.727619, 8.139442, 7.6954727, 8.010121, 4.0019975, 1.447345, 10.033835, 

In [5]:
def run_dynamic_nn(filename, first_layer_size, second_layer_size, name, features, label):
    print(name)
    X = all_visits[features]
    y = all_visits[label]

    val_mses = list()
    mses = list()

    start = time.time()
    features_min_max = preprocessing.MinMaxScaler()
    X_train, y_train, X_val, y_val, X_test, y_test = split_for_baseline_and_nn(X, y)

    model = compile_nn_weights_from_file(filename, X_train, first_layer_size, second_layer_size)

    X_train = features_min_max.fit_transform(X_train)
    X_val = features_min_max.transform(X_val)
    X_test = features_min_max.transform(X_test)

    y_val_pred = model.predict(X_val, batch_size=1)
    y_pred = model.predict(X_test, batch_size=1)
    val_mses.append(mean_squared_error(y_val_pred, y_val))
    mses.append(mean_squared_error(y_pred, y_test))
    end = time.time()
    print("Trial completed in %.2f s" % (end - start))
    
    return val_mses, mses, y_pred

In [6]:
for arch in possible_architectures_nn:
    print(arch)
    filenames = prebuilt_nn_models[arch]
    
    if arch == "64-64R":
        label = ICD8_LABEL
    elif arch == "64-64NR":
        label = ICD8OPP_LABEL
    else:
        label = ALL_VISITS_LABEL
    
    # run only the first file
    for filename in filenames:
        v_mses, mses, y_pred = run_dynamic_nn("../" + filename, layer_sizes[arch][0], layer_sizes[arch][1],
                                      filename, BASE_FEATURES, label)
        print_mse_metrics(v_mses)
        print_mse_metrics(mses)
        print(list(y_pred.reshape(-1)))
        print("\n")
        break

64-64
NNModels/64-64-0Lag-All/Dynamic-NN-AllVisits-64-64-Trial0-1562302673.171411.h5
Trial completed in 2.39 s
MSE and RMSE over 10 trials with standard deviation in parentheses
Average MSE: 0.808 (0.000)
Average RMSE: 0.899 (0.000)
MSE and RMSE over 10 trials with standard deviation in parentheses
Average MSE: 1.360 (0.000)
Average RMSE: 1.166 (0.000)
[2.1845877, 8.234965, 8.05232, 1.3380944, 6.250945, 3.9176447, 1.7810297, 10.33345, 8.603788, 8.078133, 8.09778, 8.211591, 3.9501383, 1.6909137, 10.720422, 9.096061, 8.312208, 7.548838, 8.734123, 4.2220893, 1.9507148, 10.910093, 8.824566, 8.077822, 8.04139, 8.53312, 4.2889805, 1.8870065, 10.966435, 8.56501, 8.0155945, 8.396107, 8.417823, 4.289666, 2.149795, 10.276074, 9.383752, 8.192503, 7.8124733, 8.469887, 4.6658854, 1.9246868, 10.445716, 8.863531, 8.315395, 7.6442385, 8.463815, 3.9208367, 1.4262224, 10.303887, 8.714316, 8.162027, 7.3783507, 8.116191, 4.3253317, 1.3645242, 10.759615, 8.865881, 8.439542, 8.220141, 8.33751, 4.2927217, 2.